In [ ]:
import pandas as pd


In [ ]:
import json


In [ ]:
#/Users/Franco/Desktop/PI MLOps - STEAM-20230926T220546Z-001/PI MLOps - STEAM


In [ ]:
with open('/Users/Franco/Desktop/Desarrollando Pi1/output_steam_games.json', 'r') as file:
    for line in file:
        data = json.loads(line)
        # Realiza operaciones con data


In [ ]:
nombre_archivo = 'output_steam_games.json'

# Lista para almacenar los objetos JSON
lista_json = []

# Leer el archivo línea por línea y cargar cada línea como un objeto JSON
with open(nombre_archivo, 'r') as archivo:
    for linea in archivo:
        objeto_json = json.loads(linea)
        lista_json.append(objeto_json)

# Crear un DataFrame de pandas con la lista de objetos JSON
df = pd.DataFrame(lista_json)

In [ ]:
print(df.head(10))

In [ ]:
df_sin_NaN = df.dropna()

In [ ]:
num_filas, num_columnas = df_sin_NaN.shape

# Imprimir el resultado
print(f"El DataFrame tiene {num_filas} filas y {num_columnas} columnas.")

In [ ]:
print(df_sin_NaN.head())

In [ ]:
JuegosSteamsinNaNcsv = "df_sin_NaN.csv"

# Exporta el DataFrame a un archivo CSV
df_sin_NaN.to_csv(JuegosSteamsinNaNcsv, index=False)

In [ ]:
import pandas as pd
import ast  # Importa ast

# Cargar el archivo JSON completo
reviews = []
with open('/Users/Franco/Desktop/Desarrollando Pi1/australian_user_reviews.json', 'r', encoding='utf-8') as file:
    for line in file.readlines():
        reviews.append(ast.literal_eval(line))  # Utiliza ast.literal_eval

# Crear un DataFrame a partir de los datos cargados
df_reviews = pd.DataFrame(reviews)

# Ahora puedes trabajar con df_reviews como lo necesites
df_reviews.head()  # Muestra las primeras 5 filas


#australian_user_reviews.json


In [ ]:
df_reviewsfull = df_reviews.explode('reviews', ignore_index=True)

# Desanidar las columnas anidadas dentro de 'reviews'
df_reviewsfull = pd.concat([df_reviewsfull, pd.json_normalize(df_reviewsfull['reviews'])], axis=1)

# Eliminar la columna original 'reviews'
df_reviewsfull.drop(columns='reviews', inplace=True)

In [ ]:
df_reviewsfull.head()

In [ ]:
#Con el segundo archivo cargado en DataFrame, solo nos falta 
# australian_users_items.json

In [ ]:

import pandas as pd
import ast  # Importa ast

# Cargar el archivo JSON completo
reviews1 = []
with open('/Users/Franco/Desktop/Desarrollando Pi1/australian_users_items.json', 'r', encoding='utf-8') as file:
    for line in file.readlines():
        reviews1.append(ast.literal_eval(line))  # Utiliza ast.literal_eval

# Crear un DataFrame a partir de los datos cargados
df_user_items = pd.DataFrame(reviews1)

# Ahora puedes trabajar con df_reviews como lo necesites
df_user_items.head()  # Muestra las primeras 5 filas


#australian_user_reviews.json

In [ ]:
df_reviewsfull1 = df_user_items.explode('items', ignore_index=True)

# Desanidar las columnas anidadas dentro de 'reviews'
df_reviewsfull1 = pd.concat([df_reviewsfull1, pd.json_normalize(df_reviewsfull1['items'])], axis=1)

# Eliminar la columna original 'reviews'
df_reviewsfull1.drop(columns='items', inplace=True)

df_reviewsfull1.head()

In [ ]:
# Analisis de sentimiento

In [ ]:
from textblob import TextBlob

# Función para analizar el sentimiento de una reseña
def analyze_sentiment(review):
    if isinstance(review, str):  # Verifica si 'review' es una cadena de texto
        analysis = TextBlob(review)
        # Evaluamos la polaridad de la reseña (negativo: < 0, neutral: 0, positivo: > 0)
        if analysis.sentiment.polarity < 0:
            return 0  # Sentimiento negativo
        elif analysis.sentiment.polarity == 0:
            return 1  # Sentimiento neutral
        else:
            return 2  # Sentimiento positivo
    else:
        return 1  # Si no es una cadena de texto, asignar valor neutral (1)

# Aplicar la función a la columna 'user_reviews.review' y crear la nueva columna 'sentiment_analysis'
df_reviewsfull['sentiment_analysis'] = df_reviewsfull['review'].apply(analyze_sentiment)

# Visualizar el resultado
print(df_reviewsfull[['review', 'sentiment_analysis']].head(50))



In [ ]:
df_reviewsfull.head(10)

In [ ]:
# Eliminar la columna "reviews"
df_reviewsfull.drop('review', axis=1, inplace=True)

# Visualizar el DataFrame resultante
print(df_reviewsfull.head(10))


In [ ]:
import pandas as pd

# Supongamos que tienes un DataFrame llamado 'df' que deseas guardar en un archivo CSV.
# Puedes reemplazar 'df' con el nombre de tu DataFrame.

# Especifica el nombre del archivo CSV en el que deseas guardar los datos.
nombre_archivo_csv = 'df_reviewssentiment.csv'

# Utiliza el método 'to_csv' de Pandas para guardar el DataFrame en el archivo CSV.
df_reviewsfull.to_csv(nombre_archivo_csv, index=False)  # El argumento 'index=False' evita que se guarde el índice en el archivo CSV


In [ ]:
# Terminado el Sentiment Analysis seguimos con la API

In [ ]:
import pandas as pd

# Cargar el archivo CSV en un DataFrame
df = pd.read_csv('df_sin_NaN.csv')

# Limpiar los datos en la columna 'genres' para ver solo el primer elemento por fila
df['genres'] = df['genres'].str.strip('[]').str.split(', ').str.get(0)

# Verificar el resultado
print(df['genres'].count())
df.head()


In [20]:
import pandas as pd
from datetime import datetime

# Cargar el archivo CSV en un DataFrame
df 

# Limpiar los datos y crear la columna 'horas_jugadas'
def clean_data(df):
    # Crear una copia independiente del DataFrame
    df_cleaned = df.copy()
    
    # Eliminar filas con valores no válidos en release_date
    df_cleaned = df_cleaned[df_cleaned['release_date'].str.match(r'\d{4}-\d{2}-\d{2}', na=False)] 
    
    # Convertir release_date a formato de fecha
    df_cleaned['release_date'] = pd.to_datetime(df_cleaned['release_date'], format='%Y-%m-%d', errors='coerce')
    
    # Calcular las horas jugadas para cada juego
    fecha_actual = datetime.now()
    df_cleaned['horas_jugadas'] = (fecha_actual - df_cleaned['release_date']).dt.total_seconds() / 3600
    
    return df_cleaned

# Llamar a la función para limpiar los datos y crear la columna 'horas_jugadas'
df_cleaned = clean_data(df)

# Ahora, puedes agrupar por año y género, y calcular la suma de horas jugadas para cada grupo
horas_por_año_genero = df_cleaned.groupby([df_cleaned['release_date'].dt.year, 'genres'])['horas_jugadas'].sum().reset_index()

# Ver el DataFrame con las horas jugadas por año y género
horas_por_año_genero






NameError: name 'df' is not defined

In [ ]:
# Definir el género específico que deseas consultar
genero_especifico = 'Adventure'  # Reemplaza 'Action' con el género que desees

# Filtrar el DataFrame por el género específico
genero_filtrado = horas_por_año_genero[horas_por_año_genero['genres'].str.contains(genero_especifico, case=False)]

# Encontrar el año con la suma máxima de horas jugadas para ese género
año_max_horas = genero_filtrado.loc[genero_filtrado['horas_jugadas'].idxmax(), 'release_date']

# Imprimir el año con la suma máxima de horas jugadas para el género específico
print(f"Año de lanzamiento con más horas jugadas para {genero_especifico}: {año_max_horas}")


In [ ]:
import pandas as pd

# Supongamos que tienes un DataFrame llamado 'df' que deseas guardar en un archivo CSV.
# Puedes reemplazar 'df' con el nombre de tu DataFrame.

# Especifica el nombre del archivo CSV en el que deseas guardar los datos.
nombre_archivo_csv = 'horas_por_año_genero.csv'

# Utiliza el método 'to_csv' de Pandas para guardar el DataFrame en el archivo CSV.
horas_por_año_genero.to_csv(nombre_archivo_csv, index=False)  # El argumento 'index=False' evita que se guarde el índice en el archivo CSV


In [17]:
#def PlayTimeGenre( genero : str ): LISTA

from fastapi import FastAPI
import pandas as pd

app = FastAPI()



# Cargar los datos en el DataFrame y convertir 'release_date' a objetos datetime
horas_por_año_genero = pd.read_csv('horas_por_año_genero.csv', parse_dates=['release_date'])

@app.get('/PlayTimeGenre/{genero}')
async def PlayTimeGenre(genero: str):
    # Filtrar el DataFrame por el género especificado
    genero_filtrado = horas_por_año_genero[horas_por_año_genero['genres'].str.contains(genero, case=False)]

    if genero_filtrado.empty:
        return {"error": f"No se encontraron juegos para el género {genero}."}

    # Encontrar el año con la suma máxima de horas jugadas para ese género
    año_max_horas = genero_filtrado.loc[genero_filtrado['horas_jugadas'].idxmax(), 'release_date'].year

    # Devolver el resultado en el formato especificado
    resultado = {"Año de lanzamiento con más horas jugadas para {}: ".format(genero): año_max_horas}
    return resultado




In [ ]:
df_cleaned.count

In [18]:
import nest_asyncio
nest_asyncio.apply()


In [21]:
import uvicorn

if __name__ == "__main__":
    uvicorn.run(app, host="0.0.0.0", port=8000)


INFO:     Started server process [2988]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


INFO:     127.0.0.1:55624 - "GET /PlayTimeGenre/Action HTTP/1.1" 200 OK
INFO:     127.0.0.1:55627 - "GET /sentiment_analysis/2012 HTTP/1.1" 404 Not Found
INFO:     127.0.0.1:55629 - "GET /PlayTimeGenre/Casual HTTP/1.1" 200 OK
INFO:     127.0.0.1:55629 - "GET /PlayTimeGenre/Casual HTTP/1.1" 200 OK
INFO:     127.0.0.1:55633 - "GET /PlayTimeGenre/Simulation HTTP/1.1" 200 OK
INFO:     127.0.0.1:55633 - "GET /PlayTimeGenre/Strategy HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [2988]


In [ ]:
#Continuo con la 2da funcion

In [ ]:
import pandas as pd

# Supongamos que tienes un DataFrame llamado 'df' que deseas guardar en un archivo CSV.
# Puedes reemplazar 'df' con el nombre de tu DataFrame.

# Especifica el nombre del archivo CSV en el que deseas guardar los datos.
nombre_archivo_csv = 'df_items.csv'

# Utiliza el método 'to_csv' de Pandas para guardar el DataFrame en el archivo CSV.
df_reviewsfull1.to_csv(nombre_archivo_csv, index=False)  # El argumento 'index=False' evita que se guarde el índice en el archivo CSV

In [ ]:
import pandas as pd

# Cargar los dos DataFrames
df1 = pd.read_csv('Steam_games.csv')
df2 = pd.read_csv('df_items.csv')

# Unir los DataFrames en función de la columna común 'title' y 'item_name'
df_combined = df1.merge(df2, left_on='title', right_on='item_name', how='inner')

# Agregar la columna 'release_date' al DataFrame y manejar los valores
df_combined['release_date'] = df_combined['release_date'].apply(
    lambda x: pd.to_datetime(x, format='%Y-%m-%d', errors='coerce')
)

# Limpiar los datos en la columna 'genres' para ver solo el primer elemento por fila
df_combined['genres'] = df_combined['genres'].str.strip("[]").str.split(', ').str.get(0)

# Mostrar los primeros registros para verificar los datos procesados
print(df_combined.head())



In [ ]:
df_combined.head()

In [ ]:

# Seleccionar las columnas deseadas
columns_to_keep = ["genres", "title", "release_date", "user_id", "playtime_forever"]

# Crear un nuevo DataFrame con las columnas seleccionadas
df_selected_columns = df_combined[columns_to_keep]

# Guardar el DataFrame con las columnas seleccionadas en un archivo CSV
df_selected_columns.to_csv('df_items_Steamgames_selected.csv', index=False)


In [ ]:

# Supongamos que tienes un DataFrame llamado 'df' que deseas guardar en un archivo CSV.
# Puedes reemplazar 'df' con el nombre de tu DataFrame.

# Especifica el nombre del archivo CSV en el que deseas guardar los datos.
nombre_archivo_csv = 'df_items_Steamgames.csv'

# Utiliza el método 'to_csv' de Pandas para guardar el DataFrame en el archivo CSV.
df_combined.to_csv(nombre_archivo_csv, index=False)  # El argumento 'index=False' evita que se guarde el índice en el archivo CSV

In [ ]:
import pandas as pd

# Cargar los datos limpios en un DataFrame
df_cleaned = pd.read_csv('df_items_Steamgames_selected.csv')

def calcular_horas_por_genero(genero):
    # Eliminar comillas del género (si las tiene)
    genero = genero.strip("'")

    # Filtrar el DataFrame por el género especificado
    genero_filtrado = df_cleaned[df_cleaned['genres'].str.strip("'") == genero]

    if genero_filtrado.empty:
        return {"error": f"No se encontraron juegos para el género {genero}."}

    # Encontrar el usuario con más horas jugadas para ese género
    max_horas_usuario = genero_filtrado.loc[genero_filtrado['playtime_forever'].idxmax(), 'user_id']

    # Calcular la acumulación de horas jugadas por año
    genero_filtrado['release_date'] = pd.to_datetime(genero_filtrado['release_date'])
    horas_por_año = genero_filtrado.groupby(genero_filtrado['release_date'].dt.year)['playtime_forever'].sum().reset_index()
    horas_por_año.rename(columns={'release_date': 'Año', 'playtime_forever': 'Horas'}, inplace=True)

    # Crear un diccionario con los resultados
    resultado = {
        "Usuario con más horas jugadas para {}: ".format(genero): max_horas_usuario,
        "Horas jugadas por año": horas_por_año.to_dict(orient='records')
    }
    
    return resultado


In [ ]:
import pandas as pd

# Cargar los datos limpios en un DataFrame
df_cleaned = pd.read_csv('df_items_Steamgames_selected.csv')

def calcular_horas_por_genero(genero):
    # Eliminar comillas del género (si las tiene)
    genero = genero.strip("'")

    # Filtrar el DataFrame por el género especificado
    genero_filtrado = df_cleaned[df_cleaned['genres'].str.strip("'") == genero]

    if genero_filtrado.empty:
        return {"error": f"No se encontraron juegos para el género {genero}."}

    # Encontrar el usuario con más horas jugadas para ese género
    max_horas_usuario = genero_filtrado.loc[genero_filtrado['playtime_forever'].idxmax(), 'user_id']

    # Calcular la acumulación de horas jugadas por año
    genero_filtrado['release_date'] = pd.to_datetime(genero_filtrado['release_date'])
    horas_por_año = genero_filtrado.groupby(genero_filtrado['release_date'].dt.year)['playtime_forever'].sum().reset_index()
    horas_por_año.rename(columns={'release_date': 'Año', 'playtime_forever': 'Horas'}, inplace=True)

    # Crear un diccionario con los resultados
    resultado = {
        "Usuario con más horas jugadas para {}: ".format(genero): max_horas_usuario,
        "Horas jugadas por año": horas_por_año.to_dict(orient='records')
    }
    
    return resultado


In [ ]:
genero = 'Action'  # Cambia 'Action' por el género que desees consultar
resultados = calcular_horas_por_genero(genero)
print(resultados)


In [ ]:
from fastapi import FastAPI
import pandas as pd

app = FastAPI()

# Cargar los datos limpios en un DataFrame
df_cleaned = pd.read_csv('df_items_Steamgames_selected.csv')

def calcular_horas_por_genero(genero):
    # Eliminar comillas del género (si las tiene)
    genero = genero.strip("'")

    # Filtrar el DataFrame por el género especificado
    genero_filtrado = df_cleaned[df_cleaned['genres'].str.strip("'") == genero]

    if genero_filtrado.empty:
        return {"error": f"No se encontraron juegos para el género {genero}."}

    # Encontrar el usuario con más horas jugadas para ese género
    max_horas_usuario = genero_filtrado.loc[genero_filtrado['playtime_forever'].idxmax(), 'user_id']

    # Calcular la acumulación de horas jugadas por año
    genero_filtrado['release_date'] = pd.to_datetime(genero_filtrado['release_date'])
    horas_por_año = genero_filtrado.groupby(genero_filtrado['release_date'].dt.year)['playtime_forever'].sum().reset_index()
    horas_por_año.rename(columns={'release_date': 'Año', 'playtime_forever': 'Horas'}, inplace=True)

    # Crear un diccionario con los resultados
    resultado = {
        "Usuario con más horas jugadas para {}: ".format(genero): max_horas_usuario,
        "Horas jugadas por año": horas_por_año.to_dict(orient='records')
    }
    
    return resultado

@app.get('/UserForGenre/{genero}')
async def UserForGenre(genero: str):
    resultado = calcular_horas_por_genero(genero)
    return resultado



In [ ]:
import uvicorn
import nest_asyncio

nest_asyncio.apply()

if __name__ == "__main__":
    uvicorn.run(app, host="0.0.0.0", port=8000)


In [ ]:
import uvicorn

if __name__ == "__main__":
    uvicorn.run(app, host="0.0.0.0", port=8000)

In [ ]:
#Continuare con la 3ra funcion

In [ ]:
# Supongamos que tienes un DataFrame llamado 'df' que deseas guardar en un archivo CSV.
# Puedes reemplazar 'df' con el nombre de tu DataFrame.

# Especifica el nombre del archivo CSV en el que deseas guardar los datos.
nombre_archivo_csv = 'df_reviews.csv'

# Utiliza el método 'to_csv' de Pandas para guardar el DataFrame en el archivo CSV.
df_reviewsfull.to_csv(nombre_archivo_csv, index=False)  # El argumento 'index=False' evita que se guarde el índice en el archivo CSV

In [22]:
import pandas as pd

# Cargar los archivos CSV en DataFrames
df_reviews = pd.read_csv('df_reviewssentiment.csv', usecols=['recommend', 'sentiment_analysis', 'posted', 'item_id'])
df_items = pd.read_csv('df_items.csv', usecols=['item_name', 'item_id'])

# Eliminar filas con valores NaN o duplicados en ambos DataFrames
df_reviews.dropna(inplace=True)
df_reviews.drop_duplicates(inplace=True)
df_items.dropna(inplace=True)
df_items.drop_duplicates(inplace=True)

# Verificar el resultado
print("DataFrame 'df_reviews':")
print(df_reviews.head())

print("\nDataFrame 'df_items':")
print(df_items.head())


DataFrame 'df_reviews':
                      posted   item_id recommend  sentiment_analysis
0   Posted November 5, 2011.    1250.0      True                   2
1      Posted July 15, 2011.   22200.0      True                   2
2     Posted April 21, 2011.   43110.0      True                   2
3      Posted June 24, 2014.  251610.0      True                   2
4  Posted September 8, 2013.  227300.0      True                   0

DataFrame 'df_items':
   item_id                  item_name
0     10.0             Counter-Strike
1     20.0      Team Fortress Classic
2     30.0              Day of Defeat
3     40.0         Deathmatch Classic
4     50.0  Half-Life: Opposing Force


In [ ]:
# Supongamos que tienes un DataFrame llamado 'df' que deseas guardar en un archivo CSV.
# Puedes reemplazar 'df' con el nombre de tu DataFrame.

# Especifica el nombre del archivo CSV en el que deseas guardar los datos.
nombre_archivo_csv = 'df_reviewsf3.csv'

# Utiliza el método 'to_csv' de Pandas para guardar el DataFrame en el archivo CSV.
df_reviewsfull.to_csv(nombre_archivo_csv, index=False)  # El argumento 'index=False' evita que se guarde el índice en el archivo CSV

In [24]:
import pandas as pd

# Función para filtrar las reseñas por año
def filter_reviews_by_year(df_reviews, año):
    df_reviews['posted'] = pd.to_datetime(df_reviews['posted'], format='%B %d, %Y', errors='coerce')
    df_reviews['year'] = df_reviews['posted'].dt.year

    # Filtrar las reseñas por año y las que tengan valores en 'recommend' y 'sentiment_analysis'
    juegos_del_año = df_reviews[(df_reviews['year'] == año) & (df_reviews['recommend'].notna()) & (df_reviews['sentiment_analysis'].notna())]
    
    if juegos_del_año.empty:
        print(f"No se encontraron juegos para el año {año}.")
        return juegos_del_año
    
    print(f'Reseñas filtradas por año {año}:\n{len(juegos_del_año)} reseñas')
    return juegos_del_año

# Resto del código se mantiene igual
# Función para obtener los juegos recomendados para un año específico
def get_recommended_games(juegos_del_año):
    juegos_recomendados = juegos_del_año[(juegos_del_año['recommend'] == True) & (juegos_del_año['sentiment_analysis'].isin([1, 2]))]
    print(f'Juegos recomendados:\n{len(juegos_recomendados)} juegos')
    return juegos_recomendados

# Función para encontrar los tres juegos más recomendados
def get_top_3_games(juegos_recomendados, df_items):
    top_juegos = juegos_recomendados['item_id'].value_counts().reset_index()
    top_juegos.columns = ['item_id', 'count']
    top_juegos = top_juegos.sort_values(by='count', ascending=False)
    top_3_juegos = top_juegos.head(3)
    
    def find_item_name(item_id):
        matches = df_items[df_items['item_id'] == item_id]
        return matches['item_name'].values[0] if len(matches) > 0 else 'Nombre no encontrado'
    
    top_3_juegos['item_name'] = top_3_juegos['item_id'].apply(find_item_name)
    print(f'Top 3 juegos recomendados:\n{top_3_juegos}')
    return top_3_juegos['item_name'].tolist()

# Función principal para recomendar juegos por año
def UsersRecommend(año, df_reviews, df_items):
    juegos_del_año = filter_reviews_by_year(df_reviews, año)
    
    if juegos_del_año.empty:
        return [{"error": f"No se encontraron juegos para el año {año}."}]
    
    juegos_recomendados = get_recommended_games(juegos_del_año)
    
    if juegos_recomendados.empty:
        return [{"message": f"No se encontraron juegos recomendados para el año {año}."}]
    
    top_3_juegos = get_top_3_games(juegos_recomendados, df_items)
    
    resultado = [{"Puesto {}: ".format(i + 1): juego} for i, juego in enumerate(top_3_juegos)]
    return resultado

# Ejemplo de uso
año = 2010  # Cambia el año según tus necesidades
resultado = UsersRecommend(año, df_reviews, df_items)
print(f'Resultado final:\n{resultado}')


No se encontraron juegos para el año 2010.
Resultado final:
[{'error': 'No se encontraron juegos para el año 2010.'}]


In [10]:
import pandas as pd

# Cargar los datos limpios en un DataFrame
df_reviews = pd.read_csv('df_reviews.csv')
df_items = pd.read_csv('df_items.csv')

def filter_reviews_by_year(df_reviews, año):
    df_reviews['year'] = pd.to_datetime(df_reviews['posted'], format='%B %d, %Y', errors='coerce').dt.year

    juegos_del_año = df_reviews[(df_reviews['year'] == año) & (df_reviews['recommend'] == True)]
    
    if juegos_del_año.empty:
        print(f"No se encontraron juegos para el año {año}.")
        return juegos_del_año
    
    print(f'Reseñas filtradas por año {año}:\n{len(juegos_del_año)} reseñas')
    return juegos_del_año

def get_top_3_games(juegos_del_año, df_items):
    top_juegos = juegos_del_año['item_id'].value_counts().reset_index().head(3)
    
    def find_item_name(item_id):
        matches = df_items[df_items['item_id'] == item_id]
        return matches['item_name'].values[0] if len(matches) > 0 else 'Nombre no encontrado'
    
    top_juegos['item_name'] = top_juegos['index'].apply(find_item_name)
    
    resultado = [{"Puesto {}: ".format(i + 1): juego} for i, juego in enumerate(top_juegos['item_name'].tolist())]
    return resultado

def UsersRecommend(año):
    try:
        juegos_del_año = filter_reviews_by_year(df_reviews, año)
        
        if juegos_del_año.empty:
            return [{"error": f"No se encontraron juegos para el año {año}."}]
        
        top_3_juegos = get_top_3_games(juegos_del_año, df_items)
        
        return top_3_juegos
    except Exception as e:
        return [{"error": str(e)}]

año = 2010  # Cambia el año según tus necesidades
resultado = UsersRecommend(año)
print(f'Resultado final:\n{resultado}')



No se encontraron juegos para el año 2010.
Resultado final:
[{'error': 'No se encontraron juegos para el año 2010.'}]


In [ ]:

import nest_asyncio

nest_asyncio.apply()

In [ ]:
import uvicorn

if __name__ == "__main__":
    uvicorn.run(app, host="0.0.0.0", port=8000)

In [ ]:
#Continuo con funcion 4

In [ ]:
import pandas as pd

# Función para filtrar las reseñas por año
def filter_reviews_by_year(df_reviews, año):
    df_reviews['posted'] = pd.to_datetime(df_reviews['posted'], format='%B %d, %Y', errors='coerce')
    df_reviews['year'] = df_reviews['posted'].dt.year

    # Filtrar las reseñas por año y las que tengan valores en 'recommend' y 'sentiment_analysis'
    juegos_del_año = df_reviews[(df_reviews['year'] == año) & (df_reviews['recommend'].notna()) & (df_reviews['sentiment_analysis'].notna())]
    
    if juegos_del_año.empty:
        print(f"No se encontraron juegos para el año {año}.")
        return juegos_del_año
    
    print(f'Reseñas filtradas por año {año}:\n{len(juegos_del_año)} reseñas')
    return juegos_del_año

# Función para obtener los juegos menos recomendados por usuarios para un año específico
def UsersNotRecommend(año, df_reviews, df_items):
    juegos_del_año = filter_reviews_by_year(df_reviews, año)
    
    if juegos_del_año.empty:
        return [{"message": f"No se encontraron juegos para el año {año}."}]
    
    # Filtrar las reseñas menos recomendadas por usuarios
    juegos_no_recomendados = juegos_del_año[(juegos_del_año['recommend'] == False) & (juegos_del_año['sentiment_analysis'] == 0)]
    
    if juegos_no_recomendados.empty:
        return [{"message": f"No se encontraron juegos menos recomendados por usuarios para el año {año}."}]
    
    # Contar la cantidad de juegos menos recomendados
    top_juegos = juegos_no_recomendados['item_id'].value_counts().reset_index()
    top_juegos.columns = ['item_id', 'count']
    top_juegos = top_juegos.sort_values(by='count', ascending=False)
    top_3_juegos = top_juegos.head(3)
    
    def find_item_name(item_id):
        matches = df_items[df_items['item_id'] == item_id]
        return matches['item_name'].values[0] if len(matches) > 0 else 'Nombre no encontrado'
    
    top_3_juegos['item_name'] = top_3_juegos['item_id'].apply(find_item_name)
    
    resultado = [{"Puesto {}: ".format(i + 1): juego} for i, juego in enumerate(top_3_juegos['item_name'])]
    return resultado

# Ejemplo de uso
año = 2011  # Cambia el año según tus necesidades
resultado = UsersNotRecommend(año, df_reviews, df_items)
print(f'Top 3 juegos menos recomendados por usuarios para el año {año}:\n{resultado}')



In [15]:
import pandas as pd

# Cargar los datos desde los archivos CSV
df_reviews = pd.read_csv('df_reviewssentiment.csv')
df_items = pd.read_csv('df_items.csv')

# Función para filtrar las reseñas por año
def filter_reviews_by_year(df_reviews, año):
    df_reviews['posted'] = pd.to_datetime(df_reviews['posted'], format='%B %d, %Y', errors='coerce')
    df_reviews['year'] = df_reviews['posted'].dt.year

    # Filtrar las reseñas por año y las que tengan valores en 'recommend' y 'sentiment_analysis'
    juegos_del_año = df_reviews[(df_reviews['year'] == año) & (df_reviews['recommend'].notna()) & (df_reviews['sentiment_analysis'].notna())]
    
    if juegos_del_año.empty:
        print(f"No se encontraron juegos para el año {año}.")
        return juegos_del_año
    
    print(f'Reseñas filtradas por año {año}:\n{len(juegos_del_año)} reseñas')
    return juegos_del_año

# Función para obtener los juegos menos recomendados por usuarios para un año específico
def UsersNotRecommend(año, df_reviews, df_items):
    juegos_del_año = filter_reviews_by_year(df_reviews, año)
    
    if juegos_del_año.empty:
        return [{"message": f"No se encontraron juegos para el año {año}."}]
    
    # Filtrar las reseñas menos recomendadas por usuarios
    juegos_no_recomendados = juegos_del_año[(juegos_del_año['recommend'] == False) & (df_reviews['sentiment_analysis'] == 0)]
    
    if juegos_no_recomendados.empty:
        return [{"message": f"No se encontraron juegos menos recomendados por usuarios para el año {año}."}]
    
    # Contar la cantidad de juegos menos recomendados
    top_juegos = juegos_no_recomendados['item_id'].value_counts().reset_index()
    top_juegos.columns = ['item_id', 'count']
    top_juegos = top_juegos.sort_values(by='count', ascending=False)
    top_3_juegos = top_juegos.head(3)
    
    def find_item_name(item_id):
        matches = df_items[df_items['item_id'] == item_id]
        return matches['item_name'].values[0] if len(matches) > 0 else 'Nombre no encontrado'
    
    top_3_juegos['item_name'] = top_3_juegos['item_id'].apply(find_item_name)
    
    resultado = [{"Puesto {}: ".format(i + 1): juego} for i, juego in enumerate(top_3_juegos['item_name'])]
    return resultado

# Ejemplo de uso
año = 2011  # Cambia el año según tus necesidades
resultado = UsersNotRecommend(año, df_reviews, df_items)
print(f'Top 3 juegos menos recomendados por usuarios para el año {año}:\n{resultado}')



No se encontraron juegos para el año 2011.
Top 3 juegos menos recomendados por usuarios para el año 2011:
[{'message': 'No se encontraron juegos para el año 2011.'}]


In [ ]:
from fastapi import FastAPI
from typing import List

app = FastAPI()

# Tu código de carga de datos y funciones definidas anteriormente debe estar aquí.

# Definir un modelo de respuesta
class JuegoResponseItem:
    def __init__(self, puesto: int, juego: str):
        self.puesto = puesto
        self.juego = juego

# Ruta para obtener los juegos menos recomendados por usuarios para un año dado
@app.get("/users_not_recommend/")
async def get_users_not_recommend(año: int):
    resultado = UsersNotRecommend(año, df_reviews, df_items)
    return [JuegoResponseItem(puesto=i+1, juego=juego) for i, juego in enumerate(resultado)]


In [ ]:
import uvicorn

if __name__ == "__main__":
    uvicorn.run(app, host="0.0.0.0", port=8000)

In [ ]:
#Continuo con funcion 5

In [ ]:
import pandas as pd

# Cargar los datos desde el archivo CSV al principio del código
df = pd.read_csv('df_reviewssentiment.csv')

# Función para obtener el análisis de sentimiento para todo el DataFrame
def sentiment_analysis(df):
    if df.empty:
        return {"Negative": 0, "Neutral": 0, "Positive": 0}
    
    # Contar la cantidad de registros para cada categoría de análisis de sentimiento
    conteo_sentimientos = df['sentiment_analysis'].value_counts().to_dict()
    
    # Asegurarse de tener todas las categorías (Negative, Neutral, Positive) en el resultado
    resultado = {
        "Negative": conteo_sentimientos.get(0, 0),
        "Neutral": conteo_sentimientos.get(1, 0),
        "Positive": conteo_sentimientos.get(2, 0)
    }
    
    return resultado

# Ejemplo de uso
resultado = sentiment_analysis(df)
print(f"Análisis de sentimiento para todos los años:\n{resultado}")




In [ ]:
import pandas as pd
import re

# Cargar los datos desde el archivo CSV al principio del código
df = pd.read_csv('df_reviewssentiment.csv')

# Función para obtener el análisis de sentimiento según el año de lanzamiento
def sentiment_analysis_por_año_lanzamiento(año, df):
    # Utilizar una expresión regular para extraer el año de la columna "posted"
    df['posted'] = df['posted'].str.extract(r'(\d{4})')
    
    # Convertir la columna "posted" a números enteros
    df['posted'] = pd.to_numeric(df['posted'], errors='coerce')
    
    # Filtrar las reseñas por el año de lanzamiento especificado
    reseñas_por_año_lanzamiento = df[df['posted'] == año]
    
    if reseñas_por_año_lanzamiento.empty:
        return {"Negative": 0, "Neutral": 0, "Positive": 0}
    
    # Contar la cantidad de registros en cada categoría de análisis de sentimiento
    conteo_negative = len(reseñas_por_año_lanzamiento[reseñas_por_año_lanzamiento['sentiment_analysis'] == 0])
    conteo_neutral = len(reseñas_por_año_lanzamiento[reseñas_por_año_lanzamiento['sentiment_analysis'] == 1])
    conteo_positive = len(reseñas_por_año_lanzamiento[reseñas_por_año_lanzamiento['sentiment_analysis'] == 2])
    
    resultado = {
        "Negative": conteo_negative,
        "Neutral": conteo_neutral,
        "Positive": conteo_positive
    }
    
    return resultado

# Ejemplo de uso
año_seleccionado = 2012  # Cambia el año según tus necesidades
resultado_año_lanzamiento = sentiment_analysis_por_año_lanzamiento(año_seleccionado, df)
print(f"Análisis de sentimiento para el año de lanzamiento {año_seleccionado}:\n{resultado_año_lanzamiento}")



In [ ]:
import pandas as pd
from fastapi import FastAPI

# Cargar los datos desde el archivo CSV
df = pd.read_csv('df_reviewssentiment.csv')

# Filtrar solo las filas que contienen valores de tipo cadena en la columna "posted"
df = df[df['posted'].str.match(r'^.*\d{4}.*$', na=False)]

# Utilizar una expresión regular para extraer el año de la columna "posted"
df['posted'] = df['posted'].str.extract(r'(\d{4})')

# Convertir la columna "posted" a números enteros
df['posted'] = pd.to_numeric(df['posted'], errors='coerce')

# Crear una instancia de FastAPI
app = FastAPI()

# Función para obtener el análisis de sentimiento según el año de lanzamiento
def sentiment_analysis_por_año_lanzamiento(año, df):
    # Filtrar las reseñas por el año de lanzamiento especificado
    reseñas_por_año_lanzamiento = df[df['posted'] == año]
    
    if reseñas_por_año_lanzamiento.empty:
        return {"Negative": 0, "Neutral": 0, "Positive": 0}
    
    # Contar la cantidad de registros en cada categoría de análisis de sentimiento
    conteo_negative = len(reseñas_por_año_lanzamiento[reseñas_por_año_lanzamiento['sentiment_analysis'] == 0])
    conteo_neutral = len(reseñas_por_año_lanzamiento[reseñas_por_año_lanzamiento['sentiment_analysis'] == 1])
    conteo_positive = len(reseñas_por_año_lanzamiento[reseñas_por_año_lanzamiento['sentiment_analysis'] == 2])
    
    resultado = {
        "Negative": conteo_negative,
        "Neutral": conteo_neutral,
        "Positive": conteo_positive
    }
    
    return resultado

# Endpoint para obtener el análisis de sentimiento por año de lanzamiento
@app.get('/sentiment_analysis/{year}')
async def get_sentiment_analysis(year: int):
    resultado_año_lanzamiento = sentiment_analysis_por_año_lanzamiento(year, df)
    return resultado_año_lanzamiento




In [ ]:
import uvicorn

if __name__ == "__main__":
    uvicorn.run(app, host="0.0.0.0", port=8000)

In [ ]:
pip freeze > requirements.txt